In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


**IMPORT THE REQUIRED LIBRARIES**

In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import seaborn as sns
import matplotlib.pyplot as plt
import re

**IMPORT DATASETS**

In [0]:
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MCQ Distractor Generator NLP/Test.csv")
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MCQ Distractor Generator NLP/Train.csv", sep = ',')

Data preprocessing

1. Split the distractor sentence into parts by using comma separator. excel
2.  colmns will be present - question , answer_text, dis1, dis2, dis3
3.Create corpus
Iterate over the rows for inderstanding context
perform 
bagging
stem
lemma
sequence of words
entity of words

3. Train the models
3. Understand the general NLP process and NLG techniques
4. Generate 3 distractors for each question, so train for each TRAIN distractor and generate TEST distrator.

Train
 - X = q,a
 - Y = d1 or d2 or d3
 
Test 
 - X = q,a
 - Y = d1 or d2 or d3
 
 
5. Join those 3 TEST distractors by comma's and check for submission.

In [5]:
print(train.isnull().sum(),'\n')
print(test.isnull().sum())

question       0
answer_text    0
distractor     0
dtype: int64 

question       0
answer_text    0
dtype: int64


In [0]:
# new data frame with split value columns

trainy = train['distractor'].str.split(",", n = 2, expand = True)

#----------------------------------------------------------------

# making 3 separate columns from new dataframe 

train['distractor'] = trainy[0]

train['distractor'] = trainy[1]

train['distractor'] = trainy[2]

# ---------------------------------------------------------------

# dropping old column names
train.drop(columns = ["distractor"], inplace = True)

In [7]:
trainy.head()
trainy.shape

(31499, 3)

In [8]:
train.head()
train.shape

(31499, 2)

In [0]:
# concat 2 dataframes on axis 1
new_train = pd.concat([train, trainy], axis=1)

In [10]:
# renaming columns

new_train.columns = ['question','answer','distractor_1','distractor_2','distractor_3']
new_train.head()

,question,answer,distractor_1,distractor_2,distractor_3
0,Meals can be served,in rooms at 9:00 p. m.,'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...
1,It can be inferred from the passage that,The local government can deal with the problem...,'If some tragedies occur again ',' relevant departments of the State Council s...,"'Currently ', ' the central government has es..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,'blame Tommy for his failing grades','blame Tommy for his failing grades',None
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...,'nonnative speakers should learn more idioms','there are no ways to master idioms'
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .',None,None


In [11]:
train.info()
train.describe()
train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31499 entries, 0 to 31498
Data columns (total 2 columns):
question       31499 non-null object
answer_text    31499 non-null object
dtypes: object(2)
memory usage: 492.2+ KB


(31499, 2)

In [12]:
test.info()
test.describe()
test.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13500 entries, 0 to 13499
Data columns (total 2 columns):
question       13500 non-null object
answer_text    13500 non-null object
dtypes: object(2)
memory usage: 211.0+ KB


(13500, 2)

In [13]:
new_train.info()
new_train.describe()
new_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31499 entries, 0 to 31498
Data columns (total 5 columns):
question        31499 non-null object
answer          31499 non-null object
distractor_1    31499 non-null object
distractor_2    22348 non-null object
distractor_3    13595 non-null object
dtypes: object(5)
memory usage: 1.2+ MB


(31499, 5)

In [14]:
31499 + 13500

44999

In [15]:
new_train.isnull().sum()

question            0
answer              0
distractor_1        0
distractor_2     9151
distractor_3    17904
dtype: int64

**TREAT NULL VALUES**

In [0]:
# dropping column with any null values 
# new_train = new_train.dropna(axis = 1, how ='any', inplace = True)

In [17]:
new_train.isnull().sum()

question            0
answer              0
distractor_1        0
distractor_2     9151
distractor_3    17904
dtype: int64

**REMOVE PUNCTUTATION**

In [18]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
# function to remove punctuation

def remove_punctuation(text):
    text_no_punctuation = "".join([char for char in text if char not in string.punctuation])
    return text_no_punctuation

In [0]:
# new_train['dist1_clean'] = new_train['distractor_1'].apply(lambda x:remove_punctuation(x))
# new_train['dist2_clean'] = new_train['distractor_2'].apply(lambda x:remove_punctuation(x))
# new_train['dist3_clean'] = new_train['distractor_3'].apply(lambda x:remove_punctuation(x))

**TOKENIZATION**

In [0]:
import re

# function to convert to lower case and tokenize.
def tokenize(text):
    tokens = re.split('\W+', text )
    return tokens 

In [0]:
# new_train['dist1_clean_token'] = new_train['dist1_clean'].apply(lambda x: tokenize(x.lower()))
# new_train['dist2_clean_token'] = new_train['dist2_clean'].apply(lambda x: tokenize(x.lower()))
# new_train['dist3_clean_token'] = new_train['dist3_clean'].apply(lambda x: tokenize(x.lower()))

**REMOVE STOPWORDS**

In [23]:
import gensim
from gensim import corpora
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
stopword = nltk.corpus.stopwords.words('english')

In [0]:
# function to remove stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword] 
    return text

In [0]:
# new_train['dist1_clean_token_no_sw'] = new_train['dist1_clean_token'].apply(lambda x: remove_stopwords(x))
# new_train['dist2_clean_token_no_sw'] = new_train['dist2_clean_token'].apply(lambda x: remove_stopwords(x))
# new_train['dist3_clean_token_no_sw'] = new_train['dist3_clean_token'].apply(lambda x: remove_stopwords(x))

**STEMMING OR LEMMATIZING** 

1. PERFORM ONLY ONE OF THE ABOVE
2. STEM - ROOT WORD WITHOUT TAKING CONTEXT INTO CONSIDERATION
3. LEMMA - ROOT WORD WHILE TAKING CONTEXT INTO CONSIDERATION

**STEMMING**

In [0]:
perf_stem = nltk.PorterStemmer()

In [0]:
# function to perform stemming
def stemming(toeknized_text):
    text = [perf_stem.stem(word) for word in tokenized_text]
    return text

In [0]:
# new_train['dist1_clean_token_no_sw_stem'] = new_train['dist1_clean_token_no_sw'].apply(lambda x: stemming(x))
# new_train['dist2_clean_token_no_sw_stem'] = new_train['dist2_clean_token_no_sw'].apply(lambda x: stemming(x))
# new_train['dist3_clean_token_no_sw_stem'] = new_train['dist3_clean_token_no_sw'].apply(lambda x: stemming(x))

**LEMMATIZING**

In [0]:
per_lemma = nltk.WordNetLemmatizer()

In [0]:
# function to perform lemmatization.
def lemmatizing(tokenized_tex):
    text = [perf_lemma(word) for word in tokenized_text]
    return text

In [0]:
# new_train['dist1_clean_token_no_sw_lemma'] = new_train['dist1_clean_token_no_sw'].apply(lambda x: lemmatizing(x))
# new_train['dist2_clean_token_no_sw_lemma'] = new_train['dist2_clean_token_no_sw'].apply(lambda x: lemmatizing(x))
# new_train['dist3_clean_token_no_sw_lemma'] = new_train['dist3_clean_token_no_sw'].apply(lambda x: lemmatizing(x))

**SPLITTING THE DATASET**

In [0]:
X = new_train[['question','answer']]
# y1 = new_train['dist1_clean_token_no_sw']
# y2 = new_train['dist2_clean_token_no_sw']
# y3 = new_train['dist3_clean_token_no_sw']

**MODEL BUILDING**

In [36]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[0], X.shape[1])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

**FIT THE MODEL** ON ALL THE **3 TRAIN DISTRACTORS** FOR GENERATING **3 NEW TEST DISTRACTORS**

In [0]:

model.fit(X, y1, epochs=20, batch_size=128, callbacks=callbacks_list)

In [0]:
model.fit(X, y2, epochs=20, batch_size=128, callbacks=callbacks_list)

In [0]:
model.fit(X, y3, epochs=20, batch_size=128, callbacks=callbacks_list)

In [0]:
# # generate characters
# for i in range(1000):
# 	x = numpy.reshape(pattern, (1, len(pattern), 1))
# 	x = x / float(n_vocab)
# 	prediction = model.predict(x, verbose=0)
# 	index = numpy.argmax(prediction)
# 	result = int_to_char[index]
# 	seq_in = [int_to_char[value] for value in pattern]
# 	sys.stdout.write(result)
# 	pattern.append(index)
# 	pattern = pattern[1:len(pattern)]
# print "\nDone."

In [0]:
# # Load Larger LSTM network and generate text
# import sys
# import numpy
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils
# # load ascii text and covert to lowercase
# filename = "wonderland.txt"
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()
# # create mapping of unique chars to integers, and a reverse mapping
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))
# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print "Total Characters: ", n_chars
# print "Total Vocab: ", n_vocab
# # prepare the dataset of input to output pairs encoded as integers
# seq_length = 100
# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
# 	seq_in = raw_text[i:i + seq_length]
# 	seq_out = raw_text[i + seq_length]
# 	dataX.append([char_to_int[char] for char in seq_in])
# 	dataY.append(char_to_int[seq_out])
# n_patterns = len(dataX)
# print "Total Patterns: ", n_patterns
# # reshape X to be [samples, time steps, features]
# X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# # normalize
# X = X / float(n_vocab)
# # one hot encode the output variable
# y = np_utils.to_categorical(dataY)
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# # load the network weights
# filename = "weights-improvement-47-1.2219-bigger.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# # pick a random seed
# start = numpy.random.randint(0, len(dataX)-1)
# pattern = dataX[start]
# print "Seed:"
# print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# # generate characters
# for i in range(1000):
# 	x = numpy.reshape(pattern, (1, len(pattern), 1))
# 	x = x / float(n_vocab)
# 	prediction = model.predict(x, verbose=0)
# 	index = numpy.argmax(prediction)
# 	result = int_to_char[index]
# 	seq_in = [int_to_char[value] for value in pattern]
# 	sys.stdout.write(result)
# 	pattern.append(index)
# 	pattern = pattern[1:len(pattern)]
# print "\nDone."